In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['train', 'test', 'train.csv', 'sample_submission.csv']


In [2]:
train_idx=pd.read_csv('../input/train.csv')
HEIGH=512#image.shape[1]
WIDTH=512#image.shape[2]
CHANEL=1#image.shape[3]
print(train_idx.shape)
print(train_idx[0:5])
#we treat this like unet arquitect and like a mask we replace target to 0 to 27 outputs from 0 to 1 values
NUM_CLASSES=28
y_df=[[int(i) for i in s.split()] for s in train_idx['Target']]


from skimage.transform import resize
from skimage import util 
img_size_target=512
def downsample(img):# not used
    
    if img_size_target==512:
        return img
    else:
        return resize(img, (img_size_target, img_size_target), mode='constant', preserve_range=True)
    
    
y_df_final=np.zeros((train_idx.shape[0],NUM_CLASSES)).astype(int)

for i,x in enumerate(y_df):
    for val in x:
        #print(x,val,i)    
        y_df_final[i,val]=1
print(y_df_final[2])   
print(y_df_final.shape)

(31072, 2)
                                     Id   Target
0  00070df0-bbc3-11e8-b2bc-ac1f6b6435d0     16 0
1  000a6c98-bb9b-11e8-b2b9-ac1f6b6435d0  7 1 2 0
2  000a9596-bbc4-11e8-b2bc-ac1f6b6435d0        5
3  000c99ba-bba4-11e8-b2b9-ac1f6b6435d0        1
4  001838f8-bbca-11e8-b2bc-ac1f6b6435d0       18
[0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
(31072, 28)


In [3]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train_idx, y_df_final, test_size=0.08, random_state=42)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(28586, 2)
(2486, 2)
(28586, 28)
(2486, 28)


In [9]:

    #image=np.concatenate((R.reshape(HEIGH,WIDTH,1),G.reshape(HEIGH,WIDTH,1),B.reshape(HEIGH,WIDTH,1)),axis=2)                    
    #return np.array(image,dtype="uint8")

from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img#,save_img
import matplotlib.pyplot as plt
from skimage import util

def keras_genereatorv1(Xdata_set,ydata_set,batch_size):
      while True:
            random_indexes = np.random.choice(len(Xdata_set), batch_size)
            batch_images = np.empty((batch_size,4, HEIGH, WIDTH, 3))
            batch_labels = np.empty((batch_size, 28))                        
            for x in range(batch_size):            
                for i, idx in enumerate(random_indexes):
                    path=Xdata_set.values[idx][0]
                    name="../input/train/{}_green.png".format(path)
                    G=downsample(np.array(load_img(name))) /255
                    #print(G.shape)
                    name="../input/train/{}_blue.png".format(path)
                    B=downsample(np.array(load_img(name))) /255
                    name="../input/train/{}_red.png".format(path)
                    R=downsample(np.array(load_img(name))) /255
                    name="../input/train/{}_yellow.png".format(path)
                    Y=downsample(np.array(load_img(name))) / 255
                    #mask=get_mask(image)
                    batch_images[i][0] =R.reshape(HEIGH,WIDTH,3)
                    batch_images[i][1] =G.reshape(HEIGH,WIDTH,3)
                    batch_images[i][2] =B.reshape(HEIGH,WIDTH,3)
                    batch_images[i][3] =Y.reshape(HEIGH,WIDTH,3)
                    batch_labels[i]=ydata_set[idx]
            yield batch_images,batch_labels#[r1,r2,r3,r4,r5,r6,r7,r8]
            
X,Y=next(keras_genereatorv1(X_train,y_train,10))
print(X.shape)
print(Y.shape)
    
            
    

(10, 4, 512, 512, 3)
(10, 28)


In [16]:
import keras
from keras import backend as K
import tensorflow as tf
from keras.models import Model
from keras.layers import merge,Input,Dense, concatenate, Conv2D, MaxPooling2D, UpSampling2D, Reshape, core, Dropout,Convolution2D,Activation,Flatten
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as K
from keras.utils.vis_utils import plot_model as plot
from keras.optimizers import SGD

def f1_loss(y_true, y_pred):
    
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return 1 - K.mean(f1)

def bce_f1_loss(y_true, y_pred):    
    return K.binary_crossentropy(y_true,y_pred)+f1_loss(y_true,y_pred)

In [32]:
import keras
from keras.layers import Dense, Dropout, LSTM,GRU
from keras.layers import Conv2D, Flatten,MaxPooling2D,GlobalMaxPooling2D
from keras.models import Sequential
from keras.layers.wrappers import TimeDistributed
from keras.models import Model
import numpy as np

input_shape = (4, 512, 512, 3)
model = Sequential()
model.add(TimeDistributed(Conv2D(256, (3, 3), activation='relu'), input_shape=input_shape))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
model.add(TimeDistributed(Dropout(0.2)))
model.add(TimeDistributed(Conv2D(128, (3, 3), activation='relu')))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
model.add(TimeDistributed(Dropout(0.2)))
model.add(TimeDistributed(Conv2D(64, (3, 3), activation='relu')))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
model.add(TimeDistributed(Dropout(0.2)))
model.add(TimeDistributed(GlobalMaxPooling2D()))
model.add(GRU(300, return_sequences = True))
model.add(Dropout(0.2))
model.add(GRU(150))
model.add(Dropout(0.2))
model.add(Dense(1024))
model.add(Dropout(0.2))
model.add(Dense(1024))
model.add(Dropout(0.2))
model.add(Dense(28,activation="sigmoid"))
optim=keras.optimizers.Adam(lr=0.00001)
model.compile(optimizer='Adadelta', loss='binary_crossentropy',metrics=["acc",f1_loss,"binary_crossentropy"])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_101 (TimeDi (None, 4, 510, 510, 256)  7168      
_________________________________________________________________
time_distributed_102 (TimeDi (None, 4, 255, 255, 256)  0         
_________________________________________________________________
time_distributed_103 (TimeDi (None, 4, 255, 255, 256)  0         
_________________________________________________________________
time_distributed_104 (TimeDi (None, 4, 253, 253, 128)  295040    
_________________________________________________________________
time_distributed_105 (TimeDi (None, 4, 126, 126, 128)  0         
_________________________________________________________________
time_distributed_106 (TimeDi (None, 4, 126, 126, 128)  0         
_________________________________________________________________
time_distributed_107 (TimeDi (None, 4, 124, 124, 64)   73792     
__________

In [ ]:
model.fit_generator(keras_genereatorv1(X_train,y_train,2),steps_per_epoch=10,epochs=100,
                    verbose=1,
                    validation_data=next(keras_genereatorv1(X_test,y_test,10)),
                    validation_steps=10
                   )